# Knowledge Representation on the Web -- RDFS tutorial

Here we'll learn the basics of RDFS (RDF Schema) and how to perform basic RDFS reasoning with rdflib (documentation [here](https://rdflib.readthedocs.io/en/stable/)) and owlrl (documentation [here](https://owl-rl.readthedocs.io/en/latest/)).

## Imports

owlrl is a library implementing basic RDFS and OWL reasoning on top of rdflib. We'll install and import its relevant symbols.

In [19]:
import sys
!{sys.executable} -m pip install rdflib owlrl

from rdflib import Graph, RDFS, RDF, URIRef, Literal, Namespace
from owlrl import DeductiveClosure, RDFS_Semantics

fish: Unknown command: truetrue
~/.config/fish/config.fish (line 1): 
truetrue
^
from sourcing file ~/.config/fish/config.fish
	called during startup


## Loading RDFS graphs

Your file `yourRDF.ttl` already contains a basic Knowledge Graph in RDF with some RDFS semantics

First, we are going to add some RDFS semantics, and inspect the graph as-is; this is also called the "asserted graph"

**Exercise 1** 
1. add additional triples using the RDFS semantics: have a look [here](https://www.w3.org/TR/rdf-schema/), and use domain and range, subPropertyOf, and Class, to say more about the instances in your graph
2. load yourRDF graph
3. print the classes in your graph
4. print the properties of a specific class in yourRDF graph
5. print all instances in yourRDF graph (all objects that have a type) 
6. explain what constitutes a vocabulary in RDF

In [20]:
AnimalSpace = Namespace("http://animal.com/")
animalGraph = Graph()
animalGraph.parse("./data/animalGraph.ttl")

animal = AnimalSpace.Animal

person = AnimalSpace.Person
farmer = AnimalSpace.Farmer
daphne = AnimalSpace.Daphne
feeds = AnimalSpace.feeds
feedsGrass = AnimalSpace.feedsGrass
feedsFish = AnimalSpace.feedsFish
cattle = AnimalSpace.Cattle
noCattle = AnimalSpace.Cattle

zebra = AnimalSpace.Zebra
penguin = AnimalSpace.Penguin
pig = AnimalSpace.Pig
chicken = AnimalSpace.Chicken
dog = AnimalSpace.Dog


animalGraph.add((cattle, RDFS.subClassOf, animal))
animalGraph.add((noCattle, RDFS.subClassOf, animal))

animalGraph.add((feeds, RDF.type, RDF.Property))
animalGraph.add((feedsGrass, RDFS.subPropertyOf, feeds))
animalGraph.add((feedsFish, RDFS.subPropertyOf, feeds))
animalGraph.add((person, RDF.type, RDFS.Class))
animalGraph.add((farmer, RDFS.subClassOf, person))
animalGraph.add((daphne, RDF.type, farmer))

animalGraph.add((pig, RDF.type, cattle))
animalGraph.add((penguin, RDF.type, noCattle))
animalGraph.add((chicken, RDF.type, cattle))
animalGraph.add((zebra, RDF.type, noCattle))
animalGraph.add((dog, RDF.type, noCattle))

animalGraph.add((daphne, RDFS.label, Literal("Daphne the farmer")))
animalGraph.add((farmer, feeds, cattle))
animalGraph.add((feeds, RDFS.domain, farmer))
animalGraph.add((feeds, RDFS.range, cattle))

animalGraph.add((daphne, feedsGrass, AnimalSpace.Chicken))
animalGraph.add((daphne, feedsGrass, AnimalSpace.Pig))
animalGraph.add((daphne, feedsFish, AnimalSpace.Penguin))



animalGraph.serialize("./data/newAnimalGraph.ttl")

print(len(animalGraph))

30


## RDFS inferencing

The inference engine in owlrl is triggered by `DeductiveClosure`, which computes the closure of the graph. This requires us to specify under which semantic regime we want to perform the inference (e.g. what kind of rules under the RDFS, OWL, etc. semantics we want the reasoner to produce derivations on). For RDFS semantics we use `RDFS_Semantics` as parameter. See extra options [here](https://owl-rl.readthedocs.io/en/latest/stubs/owlrl.html#module-owlrl)


**Exercise 2**
1. expand the graph through RDFS semantics inference
2. print how many triples the new graph has
3. print out the triples in your new graph and inspect them. 

In [21]:
DeductiveClosure(RDFS_Semantics).expand(animalGraph)

print("Number of triples", len(animalGraph))

animalGraph.serialize("./data/expandedAnimalGraph.ttl")

Number of triples 93


<Graph identifier=N45e46000650141e49205ec578a5d22f2 (<class 'rdflib.graph.Graph'>)>

## The explicit (asserted) graph vs the implicit (derived) graph, and RDF entailment

Asserted triples are those that are explicitly stated, while derived or inferred triples are those that are implicitly stated through the semantics of RDFS. 

**Exercise 3**

1. Write here code to generate a graph that contains **RDFS derived triples only** from yourRDF Knowledge Graph, not the asserted ones. See a clue on rdflib graph algebra [here](https://rdflib.readthedocs.io/en/stable/merging.html)
2. have a look at the inferred graph. Based on the RDFS semantics, explain for each triple the rule that was used to generate it.
3. Explain the concept RDF entailment, and the types of entailment RDFS can produce


In [22]:
assertedGraph = Graph().parse("./data/animalGraph.ttl")
expandedGraph = Graph().parse("./data/expandedAnimalGraph.ttl")

derivedGraph = expandedGraph - assertedGraph








## Assignment part 2: your own webapplication. 


**Exercise 4**
1. load ingredients.rdf and recipes.rdf in one graph. The graph contains types of individuals and types of relationships between them. Print all the classes and properties in the combined graph with the namespace `ind` and the `wtm` namespace/vocabulary (`http://purl.org/heals/food/`). 

2. extend the `ind` vocabulary (`http://purl.org/heals/ingredient/`) by creating a hierarchy of ingredients (**hint: http://purl.org/heals/ingredient/CoconutMilk rdf:subClassOf http://purl.org/heals/ingredient/PlantMilk), and make these superclasses human readable by giving them labels**) 
3. do the same for the `wtm` vocabulary: add a hierarchy of recipes as well as a hierarchy of properties (**hint: http://purl.org/heals/food/hasCookingTemperature rdf:subPropertyOf ...) 
4. print the entailed triples as we did in the previous exercise
5. give three examples of how RDF semantics could aid the chefs in your restaurant 
    
6. which properties and classes could you add to the `wtm` and `ind` vocabularies to further describe your recipe and ingredient knowledge graph, aiding the chefs in your restaurant?  


In [23]:
ingredientsrecipes = Graph()

ingredientsrecipes.parse("./data/ingredients.rdf")
ingredientsrecipes.parse("./data/recipes.rdf")

# 1. 
IND = Namespace("http://purl.org/heals/ingredient/")
WTM = Namespace("http://purl.org/heals/food/")
for s, p, o, in ingredientsrecipes.triples((None, None, None)):
    if WTM in s or IND in s:
        print(s)
    
    if WTM in p or IND in p:
        print(p)
    
    if WTM in o or IND in o:
        print(o)


http://purl.org/heals/ingredient/Oat
http://purl.org/heals/food/hasGluten
http://purl.org/heals/ingredient/AlmondMeal
http://purl.org/heals/ingredient/GreenOnion
http://purl.org/heals/ingredient/BananaBlueberryAlmondFlourMuffin
http://purl.org/heals/food/hasCookTime
http://purl.org/heals/ingredient/Water
http://purl.org/heals/ingredient/BananaBread
http://purl.org/heals/food/hasCookTime
http://purl.org/heals/ingredient/ThaiChicken
http://purl.org/heals/ingredient/Butter
http://purl.org/heals/ingredient/Bacon
http://purl.org/heals/ingredient/Pecan
http://purl.org/heals/food/substitutesFor
http://purl.org/heals/ingredient/Walnut
http://purl.org/heals/ingredient/Bacon
http://purl.org/heals/ingredient/GoldenKamutBread
http://purl.org/heals/ingredient/BeefStew
http://purl.org/heals/food/hasIngredient
http://purl.org/heals/ingredient/VegetableOil
http://purl.org/heals/ingredient/Tomato
http://purl.org/heals/food/hasGlycemicIndex
http://purl.org/heals/ingredient/BananaBlueberryAlmondFlourMuff

In [24]:

# 2. Creating hierarchy of spices and herbs (which are subclases of seasonings)
spice = IND.Spice
herb = IND.Herbs
nut = IND.Nut
sauce = IND.Sauce
condiment = IND.Condiment
baking = IND.BakingProduct



herbs_list = ["Tarragon", "Parsley", "Rosemary", "Basil", "Oregano", "Thyme", "Cilantro", ]
spices_list = ["BlackPepper", "Salt", "Paprika"]
nuts_list = ["Almond", "Walnut", "Peanut", "PumpkinSeed", "Pecan"]
baking_list = ["AllPurposeFlour", "AlmondMeal", "BakersYeast", "BakingPowder", "BakingSoda", "BrownSugar", "Butter", "CaneSugar", "CocoaPowder", "Cornstarch", "GlutenFreeFlour", "KamutFlour", "TapiocaFlour", "VanillaExtract", "Water", "WhiteSugar", "WholeWheatFlour"]
sauces_list = ["Mayonnaise", "WholeGrainMustard"]


for hb in herbs_list:
    ingredientsrecipes.add((URIRef(f"http://purl.org/heals/ingredients/{hb}"), RDFS.subClassOf, herb))

for sp in spices_list:
    ingredientsrecipes.add((URIRef(f"http://purl.org/heals/ingredients/{sp}"), RDFS.subClassOf, spice))

for nt in nuts_list:
    ingredientsrecipes.add((URIRef(f"http://purl.org/heals/ingredients/{nt}"), RDFS.subClassOf, nut))

for bk in baking_list:
    ingredientsrecipes.add((URIRef(f"http://purl.org/heals/ingredients/{bk}"), RDFS.subClassOf, baking))

for sc in sauces_list:
    ingredientsrecipes.add((URIRef(f"http://purl.org/heals/ingredients/{sc}"), RDFS.subClassOf, sauce))   


ingredientsrecipes.add((herb, RDFS.label, Literal("All types of herbs")))
ingredientsrecipes.add((spice, RDFS.label, Literal("All types of spices")))
ingredientsrecipes.add((nut, RDFS.label, Literal("All types of nuts")))
ingredientsrecipes.add((baking, RDFS.label, Literal("All types of baking products.")))
ingredientsrecipes.add((condiment, RDFS.label, Literal("All types of condiments")))

ingredientsrecipes.add((condiment, RDFS.subClassOf, WTM.Ingredient))
ingredientsrecipes.add((baking, RDFS.subClassOf, WTM.Ingredient))
ingredientsrecipes.add((spice, RDFS.subClassOf, condiment))
ingredientsrecipes.add((herb, RDFS.subClassOf, condiment))
ingredientsrecipes.add((nut, RDFS.subClassOf, condiment))
ingredientsrecipes.add((sauce, RDFS.subClassOf, condiment))

# Due to this taking a lot of time, I have decided to not include the rest of the ingredients. One could for example make  a subclass for vegetables, meats, liquids (with subclasses oil (oils), sour (vinegars), sweet (CornSyrup), salty (SoySauce)) etc. 

<Graph identifier=Nf23d189abd4246839ac8dfa4281c2977 (<class 'rdflib.graph.Graph'>)>

In [25]:
# 3. Same thing, but then for recipes

bakedGood = WTM.BakedGood
sweetBakedGood = WTM.SweetBakedGood
savoryBakedGood = WTM.SavoryBakedGood

cookedFood = WTM.CookedFood
friedFood = WTM.FriedFood
bowlFood = WTM.BowlFood


sweet_baked = ["AlmondBiscotti","BananaBlueberryAlmondFlourMuffin", "BananaBread", "Brownies", "FlourlessCoconutAndAlmondCake", "GlutenFreeCoconutCake", "GoldenKamutBread", "KamutMuffin", "KamutPancake", "WhiteBread", "WholeGrainBananaPancake"]
savory_baked = ["BakedChickenTender", "SaucyShepherdPie"]
cooked = ["BeefNilaga", "BeefStew", "BraisedBalsamicChicken", "CornedBeefHash", "PotRoastWithVegetables", "ThaiChicken"]
fried = ["SmotheredChickenBreast", "GrilledchickenKabob"]
bowl_food = ["ChickenSalad"]


for sb in sweet_baked:
    ingredientsrecipes.add((URIRef(f"http://purl.org/heals/ingredients/{sb}"), RDFS.subClassOf, sweetBakedGood))

for sab in savory_baked:
    ingredientsrecipes.add((URIRef(f"http://purl.org/heals/ingredients/{sab}"), RDFS.subClassOf, savoryBakedGood))

for ck in cooked:
    ingredientsrecipes.add((URIRef(f"http://purl.org/heals/ingredients/{ck}"), RDFS.subClassOf, cookedFood))

for fr in fried:
    ingredientsrecipes.add((URIRef(f"http://purl.org/heals/ingredients/{fr}"), RDFS.subClassOf, friedFood))

for bf in bowl_food:
    ingredientsrecipes.add((URIRef(f"http://purl.org/heals/ingredients/{bf}"), RDFS.subClassOf, bowlFood)) 

ingredientsrecipes.add((bowlFood, RDFS.subClassOf, WTM.Recipe))
ingredientsrecipes.add((friedFood, RDFS.subClassOf, WTM.Recipe))
ingredientsrecipes.add((cookedFood, RDFS.subClassOf, WTM.Recipe))
ingredientsrecipes.add((bakedGood, RDFS.subClassOf, WTM.Recipe))
ingredientsrecipes.add((sweetBakedGood, RDFS.subClassOf, bakedGood))
ingredientsrecipes.add((savoryBakedGood, RDFS.subClassOf, bakedGood))

ingredientsrecipes.add((savoryBakedGood, RDFS.label, Literal("All types of savory baked goods (with use of an oven)")))
ingredientsrecipes.add((cookedFood, RDFS.label, Literal("All types of cooked foods (in a pan with water/broth/stew)")))
ingredientsrecipes.add((friedFood, RDFS.label, Literal("All types of fried foods (in a pan with oil)")))
ingredientsrecipes.add((bowlFood, RDFS.label, Literal("All types of cold/lukewarm foods served in a bowl.")))

# Some property hierarchies.
ingredientsrecipes.add((WTM.isRecommended, RDF.type, RDF.Property))
ingredientsrecipes.add((WTM.isRecommendedForMeal, RDFS.subPropertyOf, WTM.isRecommended))
ingredientsrecipes.add((WTM.isRecommendedForCourse, RDFS.subPropertyOf, WTM.isRecommended))

ingredientsrecipes.add((WTM.foodProperty, RDF.type, RDF.Property))
ingredientsrecipes.add((WTM.hasGluten, RDFS.subPropertyOf, WTM.foodProperty))
ingredientsrecipes.add((WTM.hasGlycemicIndex, RDFS.subPropertyOf, WTM.foodProperty))
ingredientsrecipes.add((WTM.hasTexture, RDFS.subPropertyOf, WTM.foodProperty))


<Graph identifier=Nf23d189abd4246839ac8dfa4281c2977 (<class 'rdflib.graph.Graph'>)>

In [26]:
# 4. Entailed triple printing
print(f"Nr triples before expansion: {len(ingredientsrecipes)}")
ingredientsrecipes.serialize("./data/restaurant_graph_be.ttl")
DeductiveClosure(RDFS_Semantics).expand(ingredientsrecipes)
print(f"Nr triples after expansion: {len(ingredientsrecipes)}")
ingredientsrecipes.serialize("./data/restaurant_graph_ae.ttl")

prev_graph = Graph().parse("./data/restaurant_graph_be.ttl")
exp_graph = Graph().parse("./data/restaurant_graph_ae.ttl")

entailed_graph = exp_graph - prev_graph

print(len(entailed_graph))

for s,p,o in sorted(entailed_graph.triples((None, None, None))):
    print(s, p, o)

Nr triples before expansion: 1383
Nr triples after expansion: 2718
1335
http://purl.obolibrary.org/obo/FOODON_03400662 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Resource
http://purl.obolibrary.org/obo/FOODON_03400663 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Resource
http://purl.obolibrary.org/obo/FOODON_03400666 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Resource
http://purl.obolibrary.org/obo/FOODON_03400667 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Resource
http://purl.obolibrary.org/obo/FOODON_03400670 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Resource
http://purl.obolibrary.org/obo/FOODON_03400671 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Resource
http://purl.obolibrary.org/obo/FOODON_03400673 http://www.w3.org/1999/02/22-rdf-sy

5. 
- RDF semantics can be used for example to create orders from guests. Each order is then a new instance, and can be forwarded to kitchen, so that a chef can directly see what the guest ordered and can take into consideration any kind of exceptions + allergies and such.
- RDF semantics can of course also be used to list the items on the menu. The current graph could perhaps be expanded with the quantities of each ingredient. Then, if there are a large number of orders for the same meal, the chef can easily change the serving size to create exactly enough food in one batch.
- RDF semantics can also be used to create some kind of inventory/stock management system, where a chef could indicate which ingredients are almost out of stock, so that it can be forwarded to the person that takes care of the inventory. 

6. As mentioned before, the quantities of each of the ingredients used in the recipes might come in handy. Perhaps also a property such as *methodOfPreparation*, to explain how a vegetable/fruit or other ingredient can be used or prepared (especially for tropical ingredients). In line with the latter, perhaps also a *usedIn* property for the ingredient to specify in which recipes this ingredient is used. We could even extend the KR further by introducing classes like *KitchenUtensils*, describing the utensils that are used in the kitchen, such as different types of knives, kitchen machine, cutting boards et cetera. These can also be linked to the ingredients then, so the chef directly knows what to use or what the options are regarding the preparation of certain ingredients.